# LLM RAG Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various a RAG system with MLflow.

In [12]:
import os

Set OpenAI Key

In [13]:
os.environ["OPENAI_API_KEY"] = "redacted"

In [14]:
import pandas as pd

import mlflow

## Create a RAG system

Use Langchain and Chroma to create a RAG system that answers questions based on the MLflow documentation.

In [15]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [16]:
loader = WebBaseLoader("https://mlflow.org/docs/latest/index.html")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0),
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
)

## Evaluate the RAG system using `mlflow.evaluate()`

Create a simple function that runs each input through the RAG chain

In [17]:
def model(input_df):
    answer = []
    for index, row in input_df.iterrows():
        answer.append(qa(row["questions"]))

    return answer

Create an eval dataset

In [18]:
eval_df = pd.DataFrame(
    {
        "questions": [
            "What is MLflow?",
            "How to run mlflow.evaluate()?",
            "How to log_table()?",
            "How to load_table()?",
        ],
    }
)

Create a faithfulness metric

In [19]:
from mlflow.metrics.genai import faithfulness, EvaluationExample

# Create a good and bad example for faithfulness in the context of this problem
faithfulness_examples = [
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions. In Databricks, autologging is enabled by default. ",
        score=2,
        justification="The output provides a working solution, using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
    EvaluationExample(
        input="How do I disable MLflow autologging?",
        output="mlflow.autolog(disable=True) will disable autologging for all functions.",
        score=5,
        justification="The output provides a solution that is using the mlflow.autolog() function that is provided in the context.",
        grading_context={
            "context": "mlflow.autolog(log_input_examples: bool = False, log_model_signatures: bool = True, log_models: bool = True, log_datasets: bool = True, disable: bool = False, exclusive: bool = False, disable_for_unsupported_versions: bool = False, silent: bool = False, extra_tags: Optional[Dict[str, str]] = None) → None[source] Enables (or disables) and configures autologging for all supported integrations. The parameters are passed to any autologging integrations that support them. See the tracking docs for a list of supported autologging integrations. Note that framework-specific configurations set at any point will take precedence over any configurations set by this function."
        },
    ),
]

faithfulness_metric = faithfulness(model="openai:/gpt-4", examples=faithfulness_examples)
print(faithfulness_metric)

EvaluationMetric(name=faithfulness, greater_is_better=True, long_name=faithfulness, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called faithfulness based on the input and output.
A definition of faithfulness and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Faithfulness is only evaluated with the provided output and provided context, please ignore the provided input entirely when scoring faithfulness. Faithfulne

Create a relevance metric

In [20]:
from mlflow.metrics.genai import relevance, EvaluationExample


relevance_metric = relevance(model="openai:/gpt-4")
print(relevance_metric)

EvaluationMetric(name=relevance, greater_is_better=True, long_name=relevance, version=v1, metric_details=
Task:
You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called relevance based on the input and output.
A definition of relevance and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before completing the task.

Input:
{input}

Output:
{output}

{grading_context_columns}

Metric definition:
Relevance encompasses the appropriateness, significance, and applicability of the output with respect to both the input and context. Scores should reflect the extent to 

In [21]:
results = mlflow.evaluate(
    model,
    eval_df,
    model_type="question-answering",
    evaluators="default",
    predictions="result",
    extra_metrics=[faithfulness_metric, relevance_metric, mlflow.metrics.latency()],
    evaluator_config={
        "col_mapping": {
            "inputs": "questions",
            "context": "source_documents",
        }
    },
)
print(results.metrics)

2023/10/27 00:58:16 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/10/27 00:58:16 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2023/10/27 00:58:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2023/10/27 00:58:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2023/10/27 00:58:37 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2023/10/27 00:58:37 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2023/10/27 00:58:37 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2023/10/27 00:58:37 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: faithfulness
2023/10/27 00:58:42 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: relevance


{'toxicity/v1/mean': 0.00014654027836513706, 'toxicity/v1/variance': 1.3083456425462054e-11, 'toxicity/v1/p90': 0.00015041480219224468, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 5.425, 'flesch_kincaid_grade_level/v1/variance': 28.806874999999994, 'flesch_kincaid_grade_level/v1/p90': 11.57, 'ari_grade_level/v1/mean': 7.525, 'ari_grade_level/v1/variance': 32.546875, 'ari_grade_level/v1/p90': 13.830000000000002, 'faithfulness/v1/mean': 4.0, 'faithfulness/v1/variance': 3.0, 'faithfulness/v1/p90': 5.0, 'relevance/v1/mean': 3.0, 'relevance/v1/variance': 2.5, 'relevance/v1/p90': 4.7}


In [22]:
results.tables["eval_results_table"]

,questions,outputs,query,source_documents,latency,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,faithfulness/v1/score,faithfulness/v1/justification,relevance/v1/score,relevance/v1/justification
0,What is MLflow?,MLflow is an open source platform for managin...,What is MLflow?,"[{'lc_attributes': {}, 'lc_secrets': {}, 'meta...",3.147386,89,0.000145,13.7,16.2,5,The output provided by the model is completely...,5,The output provides a comprehensive answer to ...
1,How to run mlflow.evaluate()?,You can use the MLflow Python API to run Mlfl...,How to run mlflow.evaluate()?,"[{'lc_attributes': {}, 'lc_secrets': {}, 'meta...",2.885425,30,0.000142,6.6,8.3,1,The output claims that you can use the MLflow ...,2,The output provides a general statement about ...
2,How to log_table()?,log_table() is not a function in MLflow.,How to log_table()?,"[{'lc_attributes': {}, 'lc_secrets': {}, 'meta...",1.436052,11,0.000148,0.1,5.0,5,"The output claim that ""log_table() is not a fu...",4,The output directly answers the question asked...
3,How to load_table()?,"Sorry, I don't know.",How to load_table()?,"[{'lc_attributes': {}, 'lc_secrets': {}, 'meta...",1.391522,7,0.000151,1.3,0.6,5,"The output ""Sorry, I don't know"" is consistent...",1,"The output ""Sorry, I don't know"" does not prov..."
